# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Shi  ->  J. Shi  |  ['J. Shi']
J. Liu  ->  J. Liu  |  ['J. Liu']
M. Flock  ->  M. Flock  |  ['M. Flock']
F. Walter  ->  F. Walter  |  ['F. Walter']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']


Arxiv has 77 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2404.08759


extracting tarball to tmp_2404.08759...

 done.
Retrieving document from  https://arxiv.org/e-print/2404.09124


extracting tarball to tmp_2404.09124...

...

extracting tarball to tmp_2404.09791...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2404.09791/main_aa.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'authors' from 'tmp_2404.09791/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


J. Neumann  ->  J. Neumann  |  ['J. Neumann']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']


Found 101 bibliographic references in tmp_2404.09791/main_aa.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.09257-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.09257) | **Mixing is easy: new insights for cosmochemical evolution from  pre-stellar core collapse**  |
|| A. Bhandare, et al. -- incl., <mark>M. Flock</mark> |
|*Appeared on*| *2024-04-16*|
|*Comments*| *24 pages and 27 figures. Accepted for publication in A&A*|
|**Abstract**| Signposts of early planet formation are ubiquitous in sub-structured young discs. Dense, hot and high-pressure regions formed during gravitational collapse process, integral to star formation, facilitate dynamical mixing of dust within the protostellar disc. This provides an incentive to constrain the role of gas-dust interaction and resolve zones of dust concentration during star-disc formation. We explore if thermal and dynamical conditions developed during disc formation can generate gas flows that efficiently mix and transport well-coupled gas and dust components. We simulate the collapse of dusty molecular cloud cores with the hydrodynamics code PLUTO augmented with radiation transport and self-gravity. We use a 2D axisymmetric geometry and follow the azimuthal component of velocity. Dust is treated as Lagrangian particles that are subject to drag from the gas, whose motion is computed on a Eulerian grid. We consider 1, 10 and 100 micron-sized neutral spherical dust. Importantly, the equation of state accurately includes molecular hydrogen dissociation. We focus on molecular cloud core masses of 1 and 3 Msun and explore effects of initial rotation rates and cloud core sizes. Our study underlines mechanisms for early transport of dust from inner hot disc regions via the occurrence of meridional flows and outflow. The vortical flow fosters dynamical mixing and retention of dust, while thermal pressure driven outflow replenishes dust in the outer disc. Young dynamical precursors to planet-forming discs exhibit regions with complex hydrodynamical gas features and high-temperature structures. These can play a crucial role in concentrating dust for subsequent growth into protoplanets. Dust transport, especially, from sub-au scales surrounding the protostar to outer relatively cooler parts, offers an efficient pathway for thermal reprocessing during pre-stellar core collapse. [Abridged] |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.09422-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.09422) | **FEASTS Combined with Interferometry (I): Overall Properties of Diffuse  HI and Implications for Gas Accretion in Nearby Galaxies**  |
|| J. Wang, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2024-04-16*|
|*Comments*| *45 pages, 23 figures. In press at ApJ. Data will be released at the FEASTS site upon publication*|
|**Abstract**| We present a statistical study of the properties of diffuse HI in ten nearby galaxies, comparing the HI detected by the single-dish telescope FAST (FEASTS program) and the interferometer VLA (THINGS program), respectively. The THINGS' observation missed HI with a median of 23% due to the short-spacing problem of interferometry and limited sensitivity. We extract the diffuse HI by subtracting the dense HI, which is obtained from the THINGS data with a uniform flux-density threshold, from the total HI detected by FAST. Among the sample, the median diffuse-HI fraction is 34%, and more diffuse HI is found in galaxies exhibiting more prominent tidal-interaction signatures. The diffuse HI we detected seems to be distributed in disk-like layers within a typical thickness of $1\,\text{kpc}$, different from the more halo-like diffuse HI detected around NGC 4631 in a previous study. Most of the diffuse HI is cospatial with the dense HI and has a typical column density of $10^{17.7}$-$10^{20.1}\,\text{cm}^{-2}$. The diffuse and dense HI exhibits a similar rotational motion, but the former lags by a median of 25% in at least the inner disks, and its velocity dispersions are typically twice as high. Based on a simplified estimation of circum-galactic medium properties and assuming pressure equilibrium, the volume density of diffuse HI appears to be constant within each individual galaxy, implying its role as a cooling interface. Comparing with existing models, these results are consistent with a possible link between tidal interactions, the formation of diffuse HI, and gas accretion. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.09791-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.09791) | **Simulating nearby disc galaxies on the main star formation sequence I.  Bar formation and the building of the central gas reservoir**  |
|| P. Verwilghen, et al. -- incl., <mark>J. Neumann</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2024-04-16*|
|*Comments*| *22 pages, 17 figures*|
|**Abstract**| Past studies have long emphasised the key role played by galactic stellar bars in the context of disc secular evolution, via the redistribution of gas and stars, the triggering of star formation, and the formation of prominent structures such as rings and central mass concentrations. However, the exact physical processes acting on those structures, as well as the timescales associated with the building and consumption of central gas reservoirs are still not well understood. We are building a suite of hydro-dynamical RAMSES simulations of isolated, low-redshift galaxies that mimic the properties of the PHANGS sample. The initial conditions of the models reproduce the observed stellar mass, disc scale length, or gas fraction, and this paper presents a first subset of these models. Most of our simulated galaxies develop a prominent bar structure, which itself triggers central gas fuelling and the building of an over-density with a typical scale of 100-1000 pc. We confirm that if the host galaxy features an ellipsoidal component, the formation of the bar and gas fuelling are delayed. We show that most of our simulations follow a common time evolution, when accounting for mass scaling and the bar formation time. In our simulations, the stellar mass of $10^{10}$~M$_{\odot}$ seems to mark a change in the phases describing the time evolution of the bar and its impact on the interstellar medium. In massive discs (M$_{\star} \geq 10^{10}$~M$_{\odot}$), we observe the formation of a central gas reservoir with star formation mostly occurring within a restricted starburst region, leading to a gas depletion phase. Lower-mass systems (M$_{\star} < 10^{10}$~M$_{\odot}$) do not exhibit such a depletion phase, and show a more homogeneous spread of star-forming regions along the bar structure, and do not appear to host inner bar-driven discs or rings. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.08759-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.08759) | **Two-Dimensional Models of Microphysical Clouds on Hot Jupiters I: Cloud  Properties**  |
|| D. Powell, <mark>X. Zhang</mark> |
|*Appeared on*| *2024-04-16*|
|*Comments*| *accepted to Ap*|
|**Abstract**| We present a new two-dimensional, bin-scheme microphysical model of cloud formation in the atmospheres of hot Jupiters that includes the effects of longitudinal gas and cloud transport. We predict cloud particle size distributions as a function of planetary longitude and atmospheric height for a grid of hot Jupiters with equilibrium temperatures ranging from 1000-2100 K. The predicted 2D cloud distributions vary significantly from models that do not consider horizontal cloud transport and we discuss the microphysical and transport timescales that give rise to the differences in 2D versus 1D models. We find that the horizontal advection of cloud particles increases the cloud formation efficiency for nearly all cloud species and homogenizes cloud distributions across the planets in our model grid. In 2D models, certain cloud species are able to be transported and survive on the daysides of hot Jupiters in cases where 1D models would not predict the existence of clouds. We demonstrate that the depletion of condensible gas species varies as a function of longitude and atmospheric height across the planet, which impacts the resultant gas-phase chemistry. Finally, we discuss various model sensitivities including the sensitivity of cloud properties to microphysical parameters, which we find to be substantially less than the sensitivity to the atmospheric thermal structure and horizontal and vertical transport of condensible material. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.09124-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.09124) | **Discovery of a new IW And-type dwarf nova with both tilted disk and  tidal instability**  |
|| Y. Sun, et al. -- incl., <mark>J. Shi</mark>, <mark>J. Liu</mark> |
|*Appeared on*| *2024-04-16*|
|*Comments*| *12 pages, 12 figures, accepted to MNRAS*|
|**Abstract**| IW And-type dwarf novae are anomalous Z Cam stars featured with outbursts happening during standstill states, which are not expected in the standard disk instability model. The physical mechanisms for these variations remain unclear. In this study, we report the discovery of a new candidate IW And-type dwarf nova J0652+2436, identified with its frequent outbursts from the slowly rising standstill states. Luckily, the TESS observations during a long standstill state and the earlier K2 observations give a chance to find the orbital and negative superhump period in the light curve of J0652+2436, allowing the measurement of its mass ratio of 0.366. This mass ratio is marginally possible for the tidal instability to set in according to previous SPH simulations. Thus, we propose that the outbursts in J0652+2436 are likely to be caused by the growing accretion disk during standstills, in favor of the previous hypothesis of the mechanisms lying in all IW And stars. We conclude that J0652+2436 might be the first IW And star with both a precessing tilted disk and tidal instability, which will be an important laboratory for studying the accretion disk dynamics and help understand IW And phenomenon. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2404.09257.md
    + _build/html/tmp_2404.09257/./Plots/HD_disc_m1_1micron_200ppc_312_lowres.png
    + _build/html/tmp_2404.09257/./Plots/HD_disc_m1_rot2_1micron_200ppc_616_lowres.png
    + _build/html/tmp_2404.09257/./Plots/rotationalsupport_multisize_50x3ppc_t421.png
    + _build/html/tmp_2404.09257/./Plots/rotationalsupport_multisize_50x3ppc_t431.png
    + _build/html/tmp_2404.09257/./Plots/rotationalsupport_multisize_50x3ppc_t441.png
    + _build/html/tmp_2404.09257/./Plots/rotationalsupport_multisize_50x3ppc_t476.png
exported in  _build/html/2404.09422.md
    + _build/html/tmp_2404.09422/./N628.feasts.mom0.png
    + _build/html/tmp_2404.09422/./N628.feasts.mom1.png
    + _build/html/tmp_2404.09422/./N628.feasts.mom2.png
    + _build/html/tmp_2404.09422/./N628.msc.corrected.original.mom0.png
    + _build/html/tmp_2404.09422/./N628.msc.corrected.original.mom1.png
    + _build/html/tmp_2404.09422/./N628.msc.corrected.original.mom2.png
    + _build/html/tmp_240

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Mixing is easy: new insights for cosmochemical evolution from pre-stellar core collapse

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2404.09257-b31b1b.svg)](https://arxiv.org/abs/2404.09257)<mark>Appeared on: 2024-04-16</mark> -  _24 pages and 27 figures. Accepted for publication in A&A_

</div>
<div id="authors">

A. Bhandare, et al. -- incl., <mark>M. Flock</mark>

</div>
<div id="abstract">

**Abstract:** Signposts of early planet formation are ubiquitous in sub-structured young discs. Dense, hot and high-pressure regions formed during the gravitational collapse process, integral to star formation,  facilitate dynamical mixing of dust within the protostellar disc. This provides an incentive to constrain the role of gas and dust interaction and resolve potential zones of dust concentration during star and disc formation stages. We explore if the thermal and dynamical conditions developed during protostellar disc formation can generate gas flows that efficiently mix and transport the well-coupled gas and dust components. We simulate the collapse of dusty molecular cloud cores with the hydrodynamics code \texttt{PLUTO} augmented with radiation transport and self-gravity. We use a two-dimensional axisymmetric geometry and follow the azimuthal component of the velocity. The dust is treated as Lagrangian particles that are subject to drag from the gas, whose motion is computed on a Eulerian grid. We consider 1, 10 and 100 micron-sized neutral spherical dust grains. Importantly, the equation of state accurately includes molecular hydrogen dissociation. We focus on molecular cloud core masses of 1 and 3 $M_{\odot}$ and explore effects of different initial rotation rates and cloud core sizes. Our study underlines mechanisms for early transport of dust from the inner hot disc regions via the occurrence of two transient gas motions, namely, meridional flow and outflow. The vortical flow fosters dynamical mixing and retention of dust, while the thermal pressure driven outflow replenishes dust in the outer disc. Notably, these phenomena occur regardless of the initial cloud core mass, size and rotation rate. Young dynamical precursors to planet-forming discs exhibit regions with complex hydrodynamical gas features and high-temperature structures. These can play a crucial role in concentrating dust for subsequent growth into protoplanets. Dust transport, especially, from sub-au scales surrounding the protostar to the outer relatively cooler parts, offers an efficient pathway for thermal reprocessing during pre-stellar core collapse.

</div>

<div id="div_fig1">

<img src="tmp_2404.09257/./Plots/HD_disc_m1_1micron_200ppc_312_lowres.png" alt="Fig17" width="100%"/>

**Figure 17. -** 2D view of the first hydrostatic core evolved into a rotationally supported disc at 1221 years after its formation as a result of the fiducial collapse of a 1 $M_{\odot}$ pre-stellar core with an initial rotation rate of \mbox{$\Omega_\mathrm{0} = 1.77 \times 10^{-13}$ rad $\mathrm{s}^{-1}$}(same time snapshot as Fig. \ref{fig:HD-seconddisc007}). The dust size is fixed to a constant value of 1 $\muup$m. The four panels show the \mbox** a)** Stokes number, \mbox** b)** gas temperature, \mbox** c)** gas density and \mbox** d)** radial gas velocity within the inner 15 au of the 3000 au collapsing pre-stellar core. The gas velocity streamlines indicate the material falling onto the disc and the mixing within. The meridional flow exhibited in the plot between 5--10 au forms in the inner regions and travels outwards. This motion results in an outward transport and retains dust in the outer disc. (*fig:HDdisc007*)

</div>
<div id="div_fig2">

<img src="tmp_2404.09257/./Plots/HD_disc_m1_rot2_1micron_200ppc_616_lowres.png" alt="Fig23" width="100%"/>

**Figure 23. -** 2D view of a first hydrostatic core evolved into a rotationally supported disc at 1427 years after its formation as a result of the collapse of a 1 $M_{\odot}$ pre-stellar core with an initial rotation rate of \mbox{$\Omega_\mathrm{0} = 2.099 \times 10^{-13}$ rad $\mathrm{s}^{-1}$}(same time snapshot as Fig. \ref{fig:HD-seconddisc001}). The dust size is fixed to a constant value of 1 $\muup$m. The four panels show the \mbox** a)** Stokes number, \mbox** b)** gas temperature, \mbox** c)** gas density and \mbox** d)** radial gas velocity within the inner 15 au of the 3000 au collapsing pre-stellar core. The gas velocity streamlines indicate the material falling onto the disc and the mixing within the meridional flow located at around 10 au. (*fig:HDdisc001*)

</div>
<div id="div_fig3">

<img src="tmp_2404.09257/./Plots/rotationalsupport_multisize_50x3ppc_t421.png" alt="Fig7.1" width="25%"/><img src="tmp_2404.09257/./Plots/rotationalsupport_multisize_50x3ppc_t431.png" alt="Fig7.2" width="25%"/><img src="tmp_2404.09257/./Plots/rotationalsupport_multisize_50x3ppc_t441.png" alt="Fig7.3" width="25%"/><img src="tmp_2404.09257/./Plots/rotationalsupport_multisize_50x3ppc_t476.png" alt="Fig7.4" width="25%"/>

**Figure 7. -** Rotational support (\mbox{$\rho v_{\phi}^2 / 2 > 2 P_\mathrm{th}$}) at different time snapshots highlighting the merger of the first core disc and pseudo-disc around second core formed during the 1 $M_{\odot}$ pre-stellar core collapse. The topmost plot shows the presence of both discs, which eventually merge _after_ the outflow is launched. The white dashed vertical line marks the radius of the rotationally supported first core disc using the definition detailed in Sect. \ref{sec:hydrodisc}. Gas velocity streamlines show the infalling envelope, mixing within the discs and the outflowing gas. (*fig:discmerger*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2404.09257"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\url}[1]{\href{#1}{#1}}$
$\newcommand{\dodoi}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{\doeprint}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{\doarXiv}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{\hi}{{\rm H}{\textsc i}}$
$\newcommand{\angstrom}{\text{\normalfontÅ}}$
$\newcommand{◦ee}{\ensuremath{\text{\textdegree}}}$
$\newcommand{\jyb}{\rm{Jy~beam^{-1} }}$
$\newcommand{\jykms}{\rm{Jy~km~s^{-1} }}$
$\newcommand{\jybkms}{\rm{Jy~beam^{-1}~km~s^{-1} }}$
$\newcommand{\mjybi}{\rm{mJy~beam_{i}^{-1} }}$
$\newcommand{\mjyb}{\rm{mJy~beam^{-1} }}$
$\newcommand{\mjybs}{\rm{mJy~beam_{s}^{-1} }}$
$\newcommand{\jybs}{\rm{Jy~beam_{s}^{-1} }}$
$\newcommand{\mjybkms}{\rm{mJy~beam^{-1}~km~s^{-1} }}$
$\newcommand{\kms}{\rm{km~s^{-1} }}$
$\newcommand{\cmsq}{\rm{ cm^{-2} } }$
$\newcommand{\NHI}{N_{\rm HI}}$
$\newcommand{\Msun}{\rm{M_{\odot}}}$
$\newcommand\natexlab{#1}$</div>



<div id="title">

#  FEASTS Combined with Interferometry (I): Overall Properties of Diffuse $\hi$ and Implications for Gas Accretion in Nearby Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2404.09422-b31b1b.svg)](https://arxiv.org/abs/2404.09422)<mark>Appeared on: 2024-04-16</mark> -  _45 pages, 23 figures. In press at ApJ. Data will be released at the FEASTS site upon publication_

</div>
<div id="authors">

J. W. (王菁), et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present a statistical study of the properties of diffuse $\hi$ in ten nearby galaxies, comparing the $\hi$ detected by the single-dish telescope FAST (FEASTS program) and the interferometer VLA (THINGS program), respectively.The THINGS' observation missed $\hi$ with a median of 23 \% due to the short-spacing problem of interferometry and limited sensitivity.We extract the diffuse $\hi$ by subtracting the dense $\hi$ , which is obtained from the THINGS data with a uniform flux-density threshold, from the total $\hi$ detected by FAST.Among the sample, the median diffuse- $\hi$ fraction is 34 \% , and more diffuse $\hi$ is found in galaxies exhibiting more prominent tidal-interaction signatures.The diffuse $\hi$ we detected seems to be distributed in disk-like layers within a typical thickness of $1 \text{kpc}$ , different from the more halo-like diffuse $\hi$ detected around NGC 4631 in a previous study.Most of the diffuse $\hi$ is cospatial with the dense $\hi$ and has a typical column density of $10^{17.7}$ -- $10^{20.1} \text{cm}^{-2}$ .The diffuse and dense $\hi$ exhibits a similar rotational motion, but the former lags by a median of 25 \% in at least the inner disks, and its velocity dispersions are typically twice as high.Based on a simplified estimation of circum-galactic medium properties and assuming pressure equilibrium, the volume density of diffuse $\hi$ appears to be constant within each individual galaxy, implying its role as a cooling interface.Comparing with existing models, these results are consistent with a possible link between tidal interactions, the formation of diffuse $\hi$ , and gas accretion.

</div>

<div id="div_fig1">

<img src="tmp_2404.09422/./N628.feasts.mom0.png" alt="Fig15.1" width="7%"/><img src="tmp_2404.09422/./N628.feasts.mom1.png" alt="Fig15.2" width="7%"/><img src="tmp_2404.09422/./N628.feasts.mom2.png" alt="Fig15.3" width="7%"/><img src="tmp_2404.09422/./N628.msc.corrected.original.mom0.png" alt="Fig15.4" width="7%"/><img src="tmp_2404.09422/./N628.msc.corrected.original.mom1.png" alt="Fig15.5" width="7%"/><img src="tmp_2404.09422/./N628.msc.corrected.original.mom2.png" alt="Fig15.6" width="7%"/><img src="tmp_2404.09422/./N628.msc.corrected.degradedu.mom0.png" alt="Fig15.7" width="7%"/><img src="tmp_2404.09422/./N628.msc.corrected.degradedu.mom1.png" alt="Fig15.8" width="7%"/><img src="tmp_2404.09422/./N628.msc.corrected.degradedu.mom2.png" alt="Fig15.9" width="7%"/><img src="tmp_2404.09422/./N628.msc.corrected.dmom0u.png" alt="Fig15.10" width="7%"/><img src="tmp_2404.09422/./N628.msc.corrected.dmom1u.png" alt="Fig15.11" width="7%"/><img src="tmp_2404.09422/./N628.msc.corrected.dmom2u.png" alt="Fig15.12" width="7%"/><img src="tmp_2404.09422/./N628.msc.corrected.dmom1_imdu.png" alt="Fig15.13" width="7%"/>

**Figure 15. -** The moment images of the galaxy NGC 628. The sky region displayed is the full region of the FEASTS data.
** Row 1:** moment 0-2 images of FEASTS data, representing the total $\hi$ at the FAST resolution. The circle in the bottom left of the shows the size of the FAST beam.)
** Row 2:** moment 0-2 images of THINGS data, representing the dense $\hi$ at the VLA resolution. We do not show the VLA synthesis beam as it is too small compared to the field of view.
** Row 3:** moment 0-2 images of dense $\hi$ at the FAST resolution.
** Row 4:** moment 0-2 images of the total $\hi$ minus the dense $\hi$, representing the diffuse $\hi$ at the FAST resolution.
** Row 5:** the difference in $\hi$ moment-1 images between the dense $\hi$ and diffuse $hi$ at the FAST resolution (the former minus the latter).   (*fig:mom_n628*)

</div>
<div id="div_fig2">

<img src="tmp_2404.09422/./N7331.feasts.mom0.png" alt="Fig24.1" width="7%"/><img src="tmp_2404.09422/./N7331.feasts.mom1.png" alt="Fig24.2" width="7%"/><img src="tmp_2404.09422/./N7331.feasts.mom2.png" alt="Fig24.3" width="7%"/><img src="tmp_2404.09422/./N7331.mskna.corrected.original.mom0.png" alt="Fig24.4" width="7%"/><img src="tmp_2404.09422/./N7331.mskna.corrected.original.mom1.png" alt="Fig24.5" width="7%"/><img src="tmp_2404.09422/./N7331.mskna.corrected.original.mom2.png" alt="Fig24.6" width="7%"/><img src="tmp_2404.09422/./N7331.mskna.corrected.degradedu.mom0.png" alt="Fig24.7" width="7%"/><img src="tmp_2404.09422/./N7331.mskna.corrected.degradedu.mom1.png" alt="Fig24.8" width="7%"/><img src="tmp_2404.09422/./N7331.mskna.corrected.degradedu.mom2.png" alt="Fig24.9" width="7%"/><img src="tmp_2404.09422/./N7331.mskna.corrected.dmom0u.png" alt="Fig24.10" width="7%"/><img src="tmp_2404.09422/./N7331.mskna.corrected.dmom1u.png" alt="Fig24.11" width="7%"/><img src="tmp_2404.09422/./N7331.mskna.corrected.dmom2u.png" alt="Fig24.12" width="7%"/><img src="tmp_2404.09422/./N7331.mskna.corrected.dmom1_imdu.png" alt="Fig24.13" width="7%"/>

**Figure 24. -** Same as Figure \ref{fig:mom_n628}, but for the galaxy NGC 7331.  (*fig:mom_n7331*)

</div>
<div id="div_fig3">

<img src="tmp_2404.09422/./difprofu.png" alt="Fig10" width="100%"/>

**Figure 10. -** Radial profiles of diffuse $\hi$ properties.
The profiles are color coded by galaxy names as denoted in each panel, and the radius are normalized by the optical radius $R_{25}$. All profiles start from radius equal to beam FWHM of FAST.
All measurements are made at the same resolution of FEASTS  (9.1$\pm$2.5 kpc).
** a:** the column density profile.
The column densities are not corrected for projection because the inclinations in outer disks where diffuse $\hi$ dominates the total $\hi$ are highly uncertain.
The dashed horizontal line shows the rough column density threshold of selecting the dense $\hi$.
** b:** the profile of diffuse $\hi$ column densities over the total $\hi$ column densities.
** c:** the profile of projection corrected radial velocity difference between the dense and diffuse $\hi$(former minus latter) measured along major axis of galaxies (see Section \ref{sec:derive_prof}).
The projection-corrected radial velocities along the major axis approximate rotational velocities, but we warn of their uncertainties.
The solid and dashed curves are for the receding and approaching side of the disks respectively.
The dotted line marks the position of y$=$0.
** d:** similar as in Panel c, but the radial velocity difference is normalized by the radial velocity of the dense $\hi$ along the major axis.
Values between zero and unity indicate lagged rotational$\slash$orbital velocity (see Section \ref{sec:derive_prof}).
** e:** the profile of velocity dispersion. The solid and dashed lines are for the diffuse and dense $\hi$ respectively.
** f:** the profile of velocity dispersion over the projection corrected radial velocity along major axis for the diffuse $\hi$.
 (*fig:prof*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2404.09422"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\A}{A_{2}}$
$\newcommand{\tb}{t_{bar}}$
$\newcommand{\Msun}{\mbox{M}_{\sun}}$
$\newcommand{\kms}{km.s^{-1}}$</div>



<div id="title">

# Simulating nearby disc galaxies on the main star formation sequence

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2404.09791-b31b1b.svg)](https://arxiv.org/abs/2404.09791)<mark>Appeared on: 2024-04-16</mark> -  _22 pages, 17 figures_

</div>
<div id="authors">

P. Verwilghen, et al. -- incl., <mark>J. Neumann</mark>, <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Past studies have long emphasised the key role played by galactic stellar bars in the context of disc secular evolution, via the redistribution of gas and stars, the triggering of star formation, and the formation of prominent structures such as rings and central mass concentrations. However, the exact physical processes acting on those structures, as well as the timescales associated with the building and consumption of central gas reservoirs are still not well understood. We are building a suite of hydro-dynamical RAMSES simulations of isolated, low-redshift galaxies that mimic the properties of the PHANGS sample. The initial conditions of the models reproduce the observed stellar mass, disc scale length, or gas fraction, and this paper presents a first subset of these models. Most of our simulated galaxies develop a prominent bar structure, which itself triggers central gas fuelling and the building of an over-density with a typical scale of 100-1000 pc. We confirm that if the host galaxy features an ellipsoidal component, the formation of the bar and gas fuelling are delayed. We show that most of our simulations follow a common time evolution, when accounting for mass scaling and the bar formation time. In our simulations, the stellar mass of $10^{10}$ M $_{\odot}$ seems to mark a change in the phases describing the time evolution of the bar and its impact on the interstellar medium. In massive discs (M $_{\star} \geq 10^{10}$ M $_{\odot}$ ), we observe the formation of a central gas reservoir with star formation mostly occurring within a restricted starburst region, leading to a gas depletion phase. Lower-mass systems (M $_{\star} < 10^{10}$ M $_{\odot}$ ) do not exhibit such a depletion phase, and show a more homogeneous spread of star-forming regions along the bar structure, and do not appear to host inner bar-driven discs or rings. Our results seem to be supported by observations, and we briefly discuss how this new suite of simulations can help our understanding of the secular evolution of main sequence disc galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2404.09791/./figures/Gas_OS_NS_M9.5.png" alt="Fig14" width="100%"/>

**Figure 14. -** Time evolution as a function of $\tau$ of the gas surface density (top row, \textcolor{OliveGreen}{GAS}), formed stars' surface density (second row, \textcolor{blue}{NS}), old stars' surface density (third row, \textcolor{red}{OS}), and cumulative formed stars' surface density (bottom row, \textcolor{violet}{CNS}) of the four lowest stellar mass models (10$^{9.5}$\Msun). Each panel shows a box with a side length of 10 kpc. (*fig:Dens_Glob_095*)

</div>
<div id="div_fig2">

<img src="tmp_2404.09791/./figures/Fits_PHANGS_IC_16.png" alt="Fig8" width="100%"/>

**Figure 8. -** Results of the fit for different parameters of the models as a function of the stellar mass. Top panels: gas fraction (left) and stellar scale length (right). Middle panels: scale length ratio between the gas and stars (left) and scale length of the stellar bulge (right). Bottom panels: bulge index (left) and bulge mass fraction (right). Colour circles show the actual fits (see the text in Sec. \ref{sub:cont_params} for details) to the PHANGS-ALMA sample, while the selected values for our used initial conditions (IC(models)) are shown with purple stars. (*fig:PH_IC_Tot*)

</div>
<div id="div_fig3">

<img src="tmp_2404.09791/./figures/SFR_Glob_mean.png" alt="Fig2" width="100%"/>

**Figure 2. -** Comparison of the SFR stemming from the PHANGS data sample (grey crosses) and the SFR computed from the simulated galaxies (coloured squared). The colour of the squares corresponds to the different stellar masses. The red dashed line represents the star formation main sequence at $z=0$ from [Leroy, Sandstrom and Lang (2019)](). (*fig:SFR*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2404.09791"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

404  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
